In [2]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
# from groq import Groq

In [3]:
load_dotenv(override=True)

deepseek_api_key = os.getenv("GROQ_API_KEY")
if deepseek_api_key:
    print(f"DeepSeek Groq API key exists")
else:
    print("Groq DeepSeek API key doesn't exists")

DeepSeek Groq API key exists


In [4]:
MODEL = "deepseek-r1-distill-llama-70b"
client = OpenAI(base_url="https://api.groq.com/openai/v1", api_key=deepseek_api_key)

In [5]:
system_message = "You are a helpful assistant for an Airline called Fligai. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "DO NOT provide hypothetical, fictional answers for demonstration purposes to the customer."
system_message += "Always be accurate. If you don't know the answer, say so."

In [6]:
import re

def clean_content(content):
    return re.sub(r"<think>.*?</think>", "", content, flags=re.DOTALL).strip()

def clean_history(history):
    return [{"role":element.get("role"), "content": clean_content(element.get("content"))} for element in history]

In [7]:
def chat(message, history):
    cleaned_history = clean_history(history)
    print(f"Cleaned History: {cleaned_history}", end="\n\n")
    messages = [{"role": "system", "content": system_message}] + cleaned_history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model=MODEL, messages=messages)
    result = clean_content(response.choices[0].message.content)
    print(f"Result: {result}", end="\n\n")
    return result

# gr.ChatInterface(fn=chat, type="messages").launch()

In [8]:
# Making a mock flight price API response
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    if destination_city:
        city = destination_city.lower()
        return ticket_prices.get(city, "Unknown")
    else:
        return "Unknown"

# Make a addition calculator tool
def add_numbers(num1, num2):
    return num1 + num2

In [9]:
# There's a particular dictionary structure that's required to describe our function:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of the ticket to the destination city. Call this whenever you need to know the ticket price, for \
        example, when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

add_function = {
    "name": "add_numbers",
    "description": "Get the sum of two number. Call this whenever you need to know the sum of two integer numbers, for \
        example, when a customer asks 'What is the sum of one number and another number' or 'what is one number + second number'",
    "parameters": {
        "type": "object",
        "properties": {
            "num1": {
                "type": "integer",
                "description": "The first number to add"
            },
            "num2": {
                "type": "integer",
                "description": "The second number to add with the first number"
            }
        },
        "required": ["num1", "num2"],
        "additionalProperties": False
    }
}

In [10]:
# And this is included in a list of tools:
tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": add_function}]

In [11]:
def chat(message, history):
    cleaned_history = clean_history(history)
    print(f"Cleaned History: {cleaned_history}", end="\n\n")
    messages = [{"role": "system", "content": system_message}] + cleaned_history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    # print(f"Chat Response: {response}", end="\n")
    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(response)
        # print(f"messages: {messages}", end="\n")
        response = client.chat.completions.create(model=MODEL, messages=messages)
    result = clean_content(response.choices[0].message.content)
    # print(f"Result: {result}", end="\n\n")
    return result

In [12]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_calls = message.tool_calls
    for tool_call in tool_calls:
        tool_call_function = tool_call.function
        if tool_call_function.name == "get_ticket_price":
            arguments = json.loads(tool_call_function.arguments)
            city = arguments.get('destination_city')
            price = get_ticket_price(city)
            response = {
                "role": "tool",
                "content": json.dumps({"destination_city": city,"price": price}),
                "tool_call_id": tool_call.id
            }
            return response
        elif tool_call_function.name == "add_numbers":
            arguments = json.loads(tool_call_function.arguments)
            num1, num2 = arguments.get('num1'), arguments.get('num2')
            sum_value = add_numbers(num1, num2)
            response = {
                "role": "tool",
                "content": json.dumps({"num1": num1, "num2": num2, "sum_value": sum_value}),
                "tool_call_id": tool_call.id
            }
            return response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Cleaned History: []

Cleaned History: [{'role': 'user', 'content': 'Hello'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}]

Cleaned History: [{'role': 'user', 'content': 'Hello'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'user', 'content': 'what is the flight price to paris?'}, {'role': 'assistant', 'content': 'The flight price to Paris is $899.'}]

Cleaned History: [{'role': 'user', 'content': 'Hello'}, {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}, {'role': 'user', 'content': 'what is the flight price to paris?'}, {'role': 'assistant', 'content': 'The flight price to Paris is $899.'}, {'role': 'user', 'content': 'okay cool, and what is the sum of 125 + 255?'}, {'role': 'assistant', 'content': 'The sum of 125 + 255 is 380.'}]

